In [157]:
import pandas as pd


In [158]:
df = pd.read_csv('form.csv')

print(df) 

            Timestamp            Name Are you visiting for the first time?  \
0   3/31/2025 9:28:48      Rebecca Lu                                  NaN   
1    4/1/2025 7:57:16    Derek Liang                                   NaN   
2    4/1/2025 7:57:32     Logan golia                                  NaN   
3    4/1/2025 8:04:38     Daniel Kim                                   NaN   
4    4/1/2025 8:09:57      Kyle Hwang                                  NaN   
..                ...             ...                                  ...   
62  4/4/2025 22:08:58     Daniel Choi                                  NaN   
63  4/4/2025 22:35:19      audrey lee                                  NaN   
64  4/4/2025 22:38:41       Brian Seo                                  NaN   
65  4/4/2025 23:18:55  Nathanael Wang                                  NaN   
66                NaN      Jiwang Lee                                  NaN   

    House Church              Where would you like to be picked

In [202]:
class Driver:
    def __init__(self, name, amount_seats, pickup_location, service_type, plans):
        self.name = name
        self.amount_seats = amount_seats
        self.pickup_location = pickup_location
        self.service_type = service_type
        self.plans = plans

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return isinstance(other, Rider) and self.name == other.name


class Rider:
    def __init__(self, name, pickup_location, service_type, plans):
        self.name = name
        self.pickup_location = pickup_location
        self.service_type = service_type
        self.plans = plans

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return isinstance(other, Rider) and self.name == other.name

In [208]:
people_list = df["Name"]
locations_list = df["Where would you like to be picked up?"]
service_list = df["Which service are you attending?"]
is_driver_list = df["Are you a driver?"]
plans_list = df["Preferred after church plans?"]

drivers = set()
riders = set()

for i in range(len(people_list)):
    # p = (is_driver_list[i])
    # print(type(p))
    if is_driver_list[i] == "Yes!":
        # print(is_driver_list[i])
        drivers.add(Driver(people_list[i], 4, locations_list[i], service_list[i], plans_list[i]))
    else:
        riders.add(Rider(people_list[i], locations_list[i], service_list[i], plans_list[i]))

print("List of all drivers:\n")
for d in drivers:
    print(d.name)

List of all drivers:

Josh Paik
Daniel Choi
Lia Kim
Jonathan Mak
Brian Seo
Matthew Ahn
Jack Havemann
Olivia Chang
Jocelyn Lee
Oriana Tang
Jacob Deadlifts
Clay Murphy
Ellen Kang
Ric Chang


In [ ]:
'''
Change amount of seats from the default number of 4 to another amount.
'''
for driver in drivers:
    if driver.name == "Ellen Kang":
        driver.amount_seats = 6

In [ ]:
from collections import defaultdict
from itertools import combinations

def assign_riders_greedy(drivers, riders):
    print(len(riders))
    assignments = defaultdict(list)
    unassigned_riders = set(list(riders))

    # Group drivers and riders by pickup time
    riders_by_time = defaultdict(list)
    for rider in riders:
        riders_by_time[rider.service_type].append(rider)
    
    for driver in drivers:
        print(f'\nDriver {driver.name}')

        time = driver.service_type
        possible_riders = riders_by_time[time]
        print("possible riders", [person.name for person in possible_riders])


        found_set = False
        # Try forming groups of size 4 → then 3 → then 2 → then 1
        for group_size in range(driver.amount_seats, 0, -1):
            if found_set:
                break
            for group in combinations(possible_riders, group_size):
                locations = {r.pickup_location for r in group}
                if len(locations) <= 2 and all(r in unassigned_riders for r in group):
                    assignments[driver] = group
                    for r in group:
                        unassigned_riders.remove(r)
                    found_set = True
                    break  # Move to next driver
    print(f'unassigned riders: {[rider.name for rider in unassigned_riders]}')
    return assignments
x = assign_riders_greedy(drivers, riders)
print(x)

52

Driver Josh Paik
possible riders ['Grace Sowon Park ', 'Daniel Kuo', 'Derek Liang ', 'Claire Doh', 'Gabriel Ni', 'Elise Cho', 'Nathanael Wang', 'Jeff Jiang', 'helena song', 'Darius Ajebon ', 'Maya Habraken ', 'Josh Yang', 'Aaron Long', 'Benjamin Kim', 'Taeho Choe ', 'Emily Yang', 'Israel Haile', 'Karina Pan', 'Kyle Hwang', 'Aimy Pham', 'Rebecca Lu', 'David Zhu', 'chloe lim', 'Phillip Seo', 'Sam Ko ', 'Susanna Tang', 'Jeffery Huang', 'Hannah Kim', 'Grace Park', 'Christina Ko', 'Joshua Lee', 'Grace Kwon', 'Ella lu', 'Jay Jung', 'Jiwang Lee', 'Joann', 'Austin Lee', 'Hannah Zhang', 'Jacob Lei', 'Angel Koh', 'Khang Le', 'Sehyun Jung', 'Ethan Yu', 'Logan golia', 'Daniel Kim ', 'Melody Hong', 'JJ Lee', 'Seojin Kwon', 'Ellie Jung ']

Driver Daniel Choi
possible riders ['Grace Sowon Park ', 'Daniel Kuo', 'Derek Liang ', 'Claire Doh', 'Gabriel Ni', 'Elise Cho', 'Nathanael Wang', 'Jeff Jiang', 'helena song', 'Darius Ajebon ', 'Maya Habraken ', 'Josh Yang', 'Aaron Long', 'Benjamin Kim', 'Taeho

In [ ]:
for driver, riders in x.items():
    print(f"\nDriver: {driver.name} {driver.pickup_location.split()[0]}")
    for rider in riders:
        print(rider.name, rider.pickup_location.split()[0])



Driver: Josh Paik Off
Grace Sowon Park  South
Daniel Kuo South
Derek Liang  South
Claire Doh South

Driver: Daniel Choi Off
Gabriel Ni North
Elise Cho South
helena song North
Darius Ajebon  North

Driver: Lia Kim Off
Jane Yoo Off
audrey lee South
Rachel Kim South

Driver: Jonathan Mak North
Nathanael Wang Life
Jeff Jiang Off
Josh Yang Off
Aaron Long Life

Driver: Brian Seo South
Maya Habraken  South
Benjamin Kim South
Taeho Choe  South
Emily Yang North

Driver: Matthew Ahn Off
Israel Haile South
Karina Pan Off
Kyle Hwang South
Rebecca Lu Off

Driver: Jack Havemann Life
Aimy Pham North
David Zhu South
Susanna Tang North
Jeffery Huang South

Driver: Olivia Chang Life
chloe lim Life
Phillip Seo Life
Sam Ko  Off
Grace Park Life

Driver: Jocelyn Lee Off
Hannah Kim South
Christina Ko Life
Joshua Lee Life
Grace Kwon South

Driver: Jacob Deadlifts Off
Ella lu South
Jay Jung North
Jiwang Lee North
Joann South

Driver: Clay Murphy Life
Austin Lee Off
Hannah Zhang South
Jacob Lei Off
Khang Le Of